# Convolutional Neural Netowrks 

In [1]:
import math
import numpy as np
import tensorflow as tf

In [10]:
from tensorflow.examples.tutorials.mnist import input_data

In [11]:
mnist_dataset = input_data.read_data_sets("../data/mnist/", one_hot=True)

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz


In [15]:
NUM_CLASSES = mnist_dataset.train.labels.shape # Number of category

IMAGE_H = 28 # hight (pixel) of image
IMAGE_W = 28 # width (pixel) of image
IMAGE_C = 1  # channel of iamge

CONV1_C = 32 # First convolutional layer's channel size
CONV1_F = 5  # First convolutional layer's kernel szie

CONV2_C = 64 # Second convolutional layer's channel size
CONV2_F = 5  # Second convolutional layer's kernel size

FC_SIZE = 512 # Fully connected layer's number of node

## Define model structure

In [16]:
def inference(images, regularizer, is_train=False):
    
    with tf.variable_scope("layer1-conv1"):
        # Define convolution weights and biases
        conv1_weights = tf.get_variable(
            name="weight", 
            shape=[CONV1_F, CONV1_F, IMAGE_C, CONV1_C], 
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        conv1_biases = tf.get_variable(
            name="bias", 
            shape=[CONV1_C],
            initializer=tf.constant_initializer(0.0)
        )
        
        # input is a tensor that shape is [batch, in_height, in_width, in_channels]
        # filter is a tensor that shape is [filter_height, filter_width, in_channels, out_channels]
        # strides is a 1-D tensor of length 4. The stride of the sliding window for each dimension of input
        # padding is a string from: "SAME", "VALID". The type of padding algorithm to use.
        conv1 = tf.nn.conv2d(
            input=images,
            filter=conv1_weights,
            strides=[1, 1, 1, 1],
            padding="SAME"
        )
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))
    
    with tf.name_scope("layer2-pool1"):
        # value is a 4-D Tensor with shape [batch, height, width, channels] and type tf.float32.
        # ksize is a list of ints that has length >= 4. The size of the window for each dimension of the input tensor.
        # strides is a list of ints that has length >= 4. The stride of the sliding window for each dimension of the input tensor.
        # padding is a string from: "SAME", "VALID". The type of padding algorithm to use.
        pool1 = tf.nn.max_pool(
            value=relu1, 
            ksize=[1, 2, 2, 1], 
            strides=[1, 2, 2, 1], 
            padding="SAME"
        )
        
    with tf.variable_scope("layer3-conv2"):
        conv2_weights = tf.get_variable(
            name="weight",
            shape=[CONV2_F, CONV_F, CONV1_C, CONV2_C],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        conv2_biases = tf.get_variable(
            name="bias",
            shape=[CONV2_C],
            initializer=tf.constant_initializer(0.0)
        )
        conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1, 1, 1, 1], padding="SAME")
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))
    
    with tf.name_scope("layer4-pool2"):
        pool2 = tf.nn.max_pool(
            value=relu2,
            ksize=[1, 2, 2, 1],
            strides=[1, 2, 2, 1],
            padding="SAME"
        )
        
    # Resahpe layer4-pool2 output size to fit layer5-fc1 input size
    pool_shape = pool2.get_shape().as_list()
    nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]
    reshaped = tf.reshape(pool2, [pool_shape[0], nodes])
    
    with tf.variable_scope("layer5-fc1"):
        fc1_weights = tf.get_variable(
            name="weights",
            shape=[nodes, FC_SIZE],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        if regularizer is not None:
            tf.add_to_collection("losses", regularizer(fc1_weights))
        fc1_biases = tf.get_variable(
            name="bias",
            shape=[FC_SIZE],
            initializer=tf.constant_initializer(0.1)
        )
        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
        if is_train:
            fc1 = tf.nn.dropout(fc1, keep_prob=0.5)
            
    with tf.variable_scope("layer6-fc2"):
        fc2_weights = tf.get_variable(
            name="weights",
            shape=[FC_SIZE, NUM_CLASSES],
            initializer=tf.truncated_normal_initializer(stddev=0.1)
        )
        if regularizer is not None:
            tf.add_to_collection("losses", regularizer(fc2_weights))
        fc2_biases = tf.get_variable(
            name="bias",
            shape=[NUM_CLASSES],
            initializer=tf.constant_initializer(0.1)
        )
        logit = tf.matmul(fc1, fc2_weights) + fc2_biases
    
    return logit